##**Maestría en Inteligencia Artificial Aplicada**
###**Curso: Inteligencia Artificial y Aprendizaje Automático**
####Tecnológico de Monterrey
####Prof Luis Eduardo Falcón Morales

### **Adtividad de la Semana 8**
####**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Esta Actividad deberás resolverla de manera individual.**

**Nombre y matrícula:**

*   Oscar Luis Guadarrama Jimenez 
*   A01796245


In [13]:
# Agrega aquí todas las librerías y paquetes adicionales que requieras.

import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

### **Liga de Datos de la UCI "Restaurant & consumer data".**
### **Descarga "RCdata.zip" y de ahí necesitaremos los archivos "rating_final.csv" y "geoplaces2.csv".**

https://archive.ics.uci.edu/dataset/232/restaurant+consumer+data

### **Puedes también usar la siguiente liga:**

https://www.kaggle.com/datasets/prajitdatta/movielens-100k-dataset

In [14]:
# Descarga los archivos de la página de la UCI a partir de los cuales generaremos
# nuestras matrices de utilidad.

data_rating = pd.read_csv("rating_final.csv", header='infer', sep=",")
data_geo = pd.read_csv("geoplaces2.csv", header='infer',  encoding='latin-1')

print(data_rating.shape, data_geo.shape)


(1161, 5) (130, 21)


## **Ejercicio - 1**

### **Explica cuál es el propósito de usar el argumento "latin-1" al cargar el segundo archivo.**

++++++++ Inicia la sección de agregar texto: +++++++++++

El argumento "latin-1" se utiliza para que se puedan reconocer carateres de lenguas de origen latino 

++++++++ Termina la sección de agregar texto. +++++++++++

## **Ejercicio - 2**

#### **Cada restaurante debe tener un nombre diferente:**

In [15]:
# Veamos la lista de restaurantes.
# Todos ellos son restaurantes ubicados en alguna ciudad del país de México:

data_geo['name'].values

array(['Kiku Cuernavaca', 'puesto de tacos', 'El Rincón de San Francisco',
       'little pizza Emilio Portes Gil', 'carnitas_mata',
       'Restaurant los Compadres', 'Taqueria EL amigo ', 'shi ro ie',
       'Pollo_Frito_Buenos_Aires', 'la Estrella de Dimas',
       'Restaurante 75', 'Abondance Restaurante Bar',
       'El angel Restaurante', 'Restaurante Pueblo Bonito',
       'Mcdonalds Parque Tangamanga', 'Tortas y hamburguesas el gordo',
       'Sirlone', 'rockabilly ', 'Unicols Pizza', 'TACOS EL GUERO',
       'Restaurant El Muladar de Calzada', 'La Posada del Virrey',
       'Restaurant and Bar and Clothesline Carlos N Charlies', 'KFC',
       'Giovannis', 'Restaurant Oriental Express', 'Mariscos Tia Licha',
       'cafe ambar', 'Restaurante la Gran Via', 'don burguers',
       'Restaurante y Pescaderia Tampico', 'Rincon del Bife',
       'La Fontana Pizza Restaurante and Cafe',
       'Restaurante la Estrella de Dima', 'El Rincon de San Francisco',
       'Preambulo Wifi Zone 

*   ### **Todos los restaurantes que aparecen en esta lista son negocios diferentes, en el sentido de que físicamente están ubicados en diferentes ciudades de México, aunque algunos pertenezcan a una franquicia, como Vips.**

*   ### **Para los fines de este ejercicio, estaremos considerando a todos los restaurantes como diferentes. Sin embargo, en el caso de Vips, existen tres restaurantes en esta lista capturados con los nombres: VIPS, Vips y vips. Que por sus coordenadas de latitud y longitud sabemos que están ubicados en SLP, Cuernavaca y cdVictoria. Se podrían cambiar los nombres agregando la ciudad, pero como se capturaron los nombres de manera diferente con minúsculas y mayúsculas, los dejaremos por el momento de esta manera.**

*   ### **Por el momento no estaremos haciendo las recomendaciones por el lugar donde se encuentran, sino solamente por la evaluación del restaurante y otras características que no tienen que ver con la ubicación.**

*   ### **Existen otras franquicias capturadas también de manera diferente, salvo el caso de 'Gorditas Dona Tota', en el cual dos de estos restaurantes que están ubicados en Cd.Victoria fueron registrados exactamente con el mismo nombre. Para que nuestro sistema de recomendación pueda diferenciarlos, deberás cambiar el nombre de uno de ellos como se te indica a continuación.**  

### **De los dos restaurantes registrados con el nombre de 'Gorditas Dona Tota', encuentra sus índices y cambia el que tiene el mayor índice en data_geo[name] al de  'Gorditas Dona Tota 2'.**

In [16]:
# Ejercicio-2

# ************* Inlcuye aquí tu código:*****************************


# Lista de índices de los registros con el nombre de 'Gorditas Dona Tota':
indices_Dona_Tota = data_geo.loc[(data_geo['name'] == 'Gorditas Dona Tota')].index
data_geo['name'].loc[data_geo['name'].loc[(data_geo['name'] == 'Gorditas Dona Tota')].index[1]] = 'Gorditas Dona Tota 2'

# *********** Aquí termina la sección de agregar código *************

print('Verificando el cambio de nombre:\n', data_geo.loc[indices_Dona_Tota, 'name'])

Verificando el cambio de nombre:
 89       Gorditas Dona Tota
118    Gorditas Dona Tota 2
Name: name, dtype: object


C:\Users\Oscar\AppData\Local\Temp\ipykernel_41920\1810356955.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data_geo['name'].loc[data_geo['name'].loc[(data_geo['name'] == 'Gorditas Dona Tota')].index[1]] = 'Gorditas Dona Tota 2'
C:\User

## **Ejercicio - 3**

In [17]:
# Al momento tenemos los siguientes DataFrames. Del primer archivo
# tenemos la evaluación general, la de la comida y la del servicio:

data_rating.head(3)

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2


In [18]:
# Y del segundo archivo obtenemos información diversa de cada restaurante:

data_geo.head(2).T

,0,1
placeID,134999,132825
latitude,18.915421,22.147392
longitude,-99.184871,-100.983092
the_geom_meter,0101000020957F000088568DE356715AC138C0A525FC46...,0101000020957F00001AD016568C4858C1243261274BA5...
name,Kiku Cuernavaca,puesto de tacos
address,Revolucion,esquina santos degollado y leon guzman
city,Cuernavaca,s.l.p.
state,Morelos,s.l.p.
country,Mexico,mexico
fax,?,?


### **De cada uno de estos archivos selecciona y combina las variables adecuadas para obtener un nuevo DataFrame cuyas columnas sean el ID de usuario (userID), el ID del restaurante (placeID), la calificación general (rating) y el nombre del restaurante (name). A este nuevo DataFrame llamarlo df_combinado.**

In [19]:
# Ejercicio 3:

# ************* Inlcuye aquí tu código:*****************************

df_combinado = pd.merge(data_rating[['userID','placeID','rating']],data_geo[['placeID','name']],on='placeID')



# *********** Aquí termina la sección de agregar código *************


# Despleguemos la dimensión y los primeros renglones de este DataFrame:

print(df_combinado.shape)
df_combinado.head()

(1161, 4)


,userID,placeID,rating,name
0,U1077,135085,2,Tortas Locas Hipocampo
1,U1077,135038,2,Restaurant la Chalita
2,U1077,132825,2,puesto de tacos
3,U1077,135060,1,Restaurante Marisco Sam
4,U1068,135104,1,vips


## **Ejercicio - 4**

In [20]:
# Ejercicio 4:

#    Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación general (rating). La llamaremos "UtMx_rating".


# ************* Inlcuye aquí tu código:*****************************

UtMx_rating = df_combinado.pivot_table(values='rating', index='name', columns='userID', fill_value=0)


# *********** Aquí termina la sección de agregar código *************


print('Dimensión de la matriz de Utilidad:')
print('(restaurantes, usuarios) =', (UtMx_rating.shape))
UtMx_rating.head()

Dimensión de la matriz de Utilidad:
(restaurantes, usuarios) = (130, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Arrachela Grill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cabana Huasteca,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
Cafe Chaires,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cafeteria cenidet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### **Similaridades con la matriz obtenida de la Factorización SVD Truncada:**

*   ### **Aplicaremo la Factorización SVD con los 66 primeros valores singulares a la matriz de utilidad UtMx_rating.**

*   ### **El valor de 66 se seleccionó porque como verás, esta cantidad de valores singulares es suficiente para describir al menos el 95% de la varianza de la matriz de utilidad.**

*   ### **Recordemos que la factorización SVD de una matriz $A$ tiene la forma:** $A_{m\times n} = U_{m\times m}\Sigma_{m\times n}V_{n\times n}^T$

*   ### **Usaremos la función coseno como medida de similaridad**

*   ### **Buscaremos restaurantes similares al llamado  "tacos de barbacoa enfrente del Tec"**.

In [21]:
num_componentes=66  # componentes más significativas

# Obtengamos la matriz de factores latentes de los restaurantes:
SVD_rating = TruncatedSVD(n_components=num_componentes)  # inicializamos
mat_fact_lat_restaurantes = SVD_rating.fit_transform(UtMx_rating) # SVD truncada-ndarray

print('Dimensión Vectores latentes restaurantes:', mat_fact_lat_restaurantes.shape)
print('Dimensión Traspuesta Vectores latentes usuarios:', SVD_rating.components_.shape)

# Calculemos la variabilidad acumulada de las componentes utilizadas:
print('\nVariabilidad acumulada de las componentes utilizadas: %.3f' % SVD_rating.explained_variance_ratio_[0:num_componentes].sum())

# matriz de similaridad
sim_matrix = cosine_similarity(mat_fact_lat_restaurantes)

# Restaurante de referencia:
restaurante_de_referencia = "tacos de barbacoa enfrente del Tec"
nombres_rest = UtMx_rating.T.columns  # lista de los nombres de los restaurantes.
idx_rest = list(nombres_rest).index(restaurante_de_referencia) # índice del restaurante de referencia.
sim_vector = sim_matrix[idx_rest] # Vector de similaridad del restaurante de referencia contra todos.
# Buscando las similaridades positivas:
idx = (sim_vector>0)
mejores_sim_rate = list()
for i in range(len(nombres_rest[idx])):
  mejores_sim_rate.append((sim_vector[idx][i], nombres_rest[idx][i]))

print('\nTotal de similaridades positivas encontradas:', len(mejores_sim_rate))

# Las ordenamos de mayor a menor relevancia:
mejores_sim_rate_ordenadas = sorted(mejores_sim_rate, key=lambda x:x[0], reverse=True)

print('\nMejores recomendaciones con base al restaurante: \"%s\":' % restaurante_de_referencia)
mejores_sim_rate_ordenadas[1:11]   # omitimos el primero que es el mismo restaurante de referencia


Dimensión Vectores latentes restaurantes: (130, 66)
Dimensión Traspuesta Vectores latentes usuarios: (66, 138)

Variabilidad acumulada de las componentes utilizadas: 0.952

Total de similaridades positivas encontradas: 74

Mejores recomendaciones con base al restaurante: "tacos de barbacoa enfrente del Tec":


[(np.float64(0.9366791494631511), 'vips'),
 (np.float64(0.9323794314398947), 'little pizza Emilio Portes Gil'),
 (np.float64(0.9240562644526806), 'tacos abi'),
 (np.float64(0.5293354484112126), 'Carreton de Flautas y Migadas'),
 (np.float64(0.5007189065197044), 'puesto de gorditas'),
 (np.float64(0.4753163416480239), 'Taqueria EL amigo '),
 (np.float64(0.3626595996142582), 'carnitas_mata'),
 (np.float64(0.2629695206481324), 'Little Cesarz'),
 (np.float64(0.23296450903032245), 'Gorditas Dona Tota 2'),
 (np.float64(0.1816864907268077), 'carnitas mata calle Emilio Portes Gil')]

## **Ejercicio - 5**

### **Sistema Híbrido**

### **Utilicemos ahora un sistema híbrido, el cual consistirá en conjuntar la matriz de "factores_latentes_restaurantes" (que es un ndarray) obtenida previamente, con la matriz de factores adicionales (que también será un ndarray) formada por las columnas One-Hot-Encoder de los factores 'dress_code', 'accessibility', 'price' y 'franchise' del DataFrame data_geo.**

### **Por el momento no requerimos que la matriz híbrida sea un DataFrame, con que sea un arreglo de NumPy, ndarray, será suficiente.**

### **Los renglones de la matriz híbrida siguen representando a cada restaurante, pero ahora con la información de los vectores latentes de la factorización SVD, más los vectores OneHotEncoding de los nuevos factores que se incluyeron.**



In [ ]:
# Ejercicio 5:

# Define el DataFrame "df_fact_adicionales" formada por los factores 'dress_code',
# 'accessibility', 'price' y 'franchise'.
# Después generar la matriz (ndarray), "mat_fact_adicionales_ohe", que se obtiene
# al tranformar las columnas (variables categóricas nominales) de "df_fact_adicionales"
# mediante OneHotEncoding.
# Finalmente conjuntar horizontalmente la matriz "mat_fact_lat_restaurantes"
# con "mat_fact_adicionales_ohe" para obtener la "matriz_hibrida" (ndarray).
# Toma en cuenta que "matriz_hibrida" estará formada por las columnas de "mat_fact_lat_restaurantes"
# seguida por las columnas de "mat_fact_adicionales_ohe".


# ************* Inlcuye aquí tu código:*****************************

df_fact_adicionales = data_geo[['name','dress_code','accessibility', 'price' ,'franchise']].sort_values(by='name').reset_index(drop= True).drop(columns='name') #Creacion de un df ordenado por nombres debido a que la matriz con la que que se va a unir esta ordenado por nombres. 
mat_fact_adicionales_ohe = OneHotEncoder().fit_transform(df_fact_adicionales).toarray() #Creacion de la matriz a concatenar, apartir del df fact addiconales, y convertido en array 
matriz_hibrida = np.concat((mat_fact_lat_restaurantes,mat_fact_adicionales_ohe),axis=1) #Concatenacion de la matriz "mat_fact_lat_restaurantes" con la matriz "mat_fact_adicionales_ohe"


# *********** Aquí termina la sección de agregar código *************


print('Dimensión de la matriz híbrida:')
matriz_hibrida.shape

Dimensión de la matriz híbrida:


(130, 77)

In [23]:
# Recordemos que "restaurante_de_referencia" es "tacos de barbacoa enfrente del Tec".
# Obtener el índice del restaurante
idx = data_geo[data_geo['name'] == restaurante_de_referencia].index[0]

print('Restaurante de referencia:', restaurante_de_referencia)
print('Índice del restaurante de referencia:', idx)

Restaurante de referencia: tacos de barbacoa enfrente del Tec
Índice del restaurante de referencia: 111


## **Ejercicio - 6:**

### **Una vez que tenemos la matriz híbrida, con dicha matriz deberás obtener las similitudes (con respecto a la función coseno) del restaurante de referencia con relación a todos los restaurantes, ordenarlos de mayor a menor con respecto a la similitud del coseno y desplegar los primeros 10 de mayor similitud (sin incluir al restaurante de referencia mismo).**

In [24]:
# Ejercicio 6

# ************* Inlcuye aquí tu código:*****************************

sim_matrix = cosine_similarity(matriz_hibrida)

restaurante_de_referencia = "tacos de barbacoa enfrente del Tec"
nombres_rest = UtMx_rating.T.columns  
idx_rest = list(nombres_rest).index(restaurante_de_referencia)

sim_vector = sim_matrix[idx_rest]
idx = np.where(sim_vector > 0)[0] 
mejores_sim_rate = list()
for i in idx:
    if i != idx_rest:# Excluir el restaurante de referencia
      mejores_sim_rate.append((sim_vector[i], nombres_rest[i]))
mejores_sim_rate_ordenadas = sorted(mejores_sim_rate, key=lambda x: x[0], reverse=True)
print('\nTotal de similaridades positivas encontradas:', len(mejores_sim_rate))
print("\nTop 10 restaurantes más similares a \"%s\":" % restaurante_de_referencia)
for sim in mejores_sim_rate_ordenadas[:10]:
    print(f"Restaurante: {sim[1]}, Similitud: {sim[0]:.4f}")


# *********** Aquí termina la sección de agregar código *************


Total de similaridades positivas encontradas: 128

Top 10 restaurantes más similares a "tacos de barbacoa enfrente del Tec":
Restaurante: tacos abi, Similitud: 0.9727
Restaurante: little pizza Emilio Portes Gil, Similitud: 0.8119
Restaurante: Carnitas Mata  Calle 16 de Septiembre, Similitud: 0.7692
Restaurante: Carreton de Flautas y Migadas, Similitud: 0.7358
Restaurante: vips, Similitud: 0.6912
Restaurante: puesto de gorditas, Similitud: 0.6693
Restaurante: cafe ambar, Similitud: 0.6680
Restaurante: Hamburguesas saul, Similitud: 0.6568
Restaurante: sirloin stockade, Similitud: 0.6358
Restaurante: Taqueria EL amigo , Similitud: 0.5970


## **Ejercicio - 7**

### **Incluye tus comentarios y conclusiones de la actividad. En particular comenta las diferencias que encuentras entre un sistema de recomendación basado solamente en la factorización SVD y uno híbrido.**

++++++++ Inicia la sección de agregar texto: +++++++++++

En esta actividad, implementaste y analizaste un sistema de recomendación basado en Descomposición en Valores Singulares (SVD) y un enfoque híbrido, integrando información adicional de los restaurantes como : dress_code, accessibility, price, franchise. 

Por un lado logramos obsevrar que Un sistema basado solo en SVD captura patrones de comportamiento, pero puede perder información relevante sobre los restaurantes, mientras que un sistema híbrido equilibra la personalización basada en usuarios con características de los ítems, ofreciendo recomendaciones más precisas y diversas.

Los resultados muestran que, aunque algunos restaurantes se mantienen en el top de recomendaciones en ambos enfoques, las similitudes cambiaron al considerar información adicional en el modelo híbrido. Esto indica que los atributos del restaurante pueden influir significativamente en las recomendaciones.

El uso de SVD junto con información adicional mejora la capacidad del sistema de recomendación, proporcionando sugerencias más alineadas con las preferencias y características del usuario. Este enfoque híbrido es más robusto, ya que no solo analiza patrones de calificación, sino que también considera mas atributos 



++++++++ Termina la sección de agregar texto. +++++++++++

>> **Fin de la Actividad de Sistemas de Recomendación**